In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
full_data = pd.read_csv("../../data/processed/historical/train/lcl_data.csv")
df_25K = full_data.sample(25000, random_state=0)
df_25K.to_csv("../../data/processed/historical/train/lcl_data_25K.csv", index=False)

# Load Data

In [3]:
import torch
import numpy as np
import random
RANDOM_STATE = 0
torch.manual_seed(RANDOM_STATE)
torch.use_deterministic_algorithms(True)
g = torch.Generator()
g.manual_seed(RANDOM_STATE)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [4]:
from pathlib import Path
from opensynth.data_modules.lcl_data_module import LCLDataModule
import pytorch_lightning as pl

import matplotlib.pyplot as plt

data_path = Path("../../data/processed/historical/train/lcl_data_25K.csv")
stats_path = Path("../../data/processed/historical/train/mean_std.csv")
outlier_path = Path("../../data/processed/historical/train/outliers.csv")

dm = LCLDataModule(data_path=data_path, stats_path=stats_path, batch_size=25000, n_samples=25000)
dm.setup()

In [5]:
import torch
from opensynth.models.faraday import FaradayVAE
vae_model = torch.load("vae_model.pt")
vae_model.eval()

/var/folders/w2/2x1gj41j1xb5z153q5m_4rn80000gn/T/ipykernel_99151/2311655816.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vae_model = torch.load("vae_model.pt")


FaradayVAE(
  (encoder): Encoder(
    (encoder_layers): Sequential(
      (0): Linear(in_features=50, out_features=512, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=512, out_features=256, bias=True)
      (3): GELU(approximate='none')
      (4): Linear(in_features=256, out_features=128, bias=True)
      (5): GELU(approximate='none')
      (6): Linear(in_features=128, out_features=64, bias=True)
      (7): GELU(approximate='none')
      (8): Linear(in_features=64, out_features=32, bias=True)
      (9): GELU(approximate='none')
      (10): Linear(in_features=32, out_features=16, bias=True)
    )
  )
  (decoder): Decoder(
    (latent): Linear(in_features=18, out_features=16, bias=True)
    (latent_activations): GELU(approximate='none')
    (decoder_layers): Sequential(
      (0): Linear(in_features=16, out_features=32, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=32, out_features=64, bias=True)
      (3): GELU(approximate='no

In [6]:
from opensynth.models.faraday.gaussian_mixture.prepare_gmm_input import encode_data_for_gmm

next_batch = next(iter(dm.train_dataloader()))
input_tensor = encode_data_for_gmm(data=next_batch, vae_module=vae_model)
input_data = input_tensor.detach().numpy()
n_samples = len(input_tensor)

In [7]:
N_COMPONENTS = 250
REG_COVAR = 1e-4
EPOCHS = 25
IDX = 0
CONVERGENCE_TOL = 1e-2


In [8]:
input_tensor[0][0]

tensor(0.0195, grad_fn=<SelectBackward0>)

# Init GMM

In [9]:
from opensynth.models.faraday.new_gmm import gmm_utils

labels_, means_, responsibilities_ = gmm_utils.initialise_centroids(
        X=input_data, n_components=N_COMPONENTS
    )
print(labels_.dtype, responsibilities_.dtype, means_.dtype)

torch.float32 torch.float32 torch.float32


In [10]:
from opensynth.models.faraday.new_gmm.train_gmm import initialise_gmm_params

gmm_init_params = initialise_gmm_params(
    X=input_data,
    n_components = N_COMPONENTS,
    reg_covar=REG_COVAR,
)
print(gmm_init_params["precision_cholesky"][IDX][0][0])
print(gmm_init_params["weights"].sum())

tensor(4.1549)
tensor(1.)


# Torch GMM

In [11]:
from opensynth.models.faraday.new_gmm.train_gmm import initialise_gmm_params, training_loop
from opensynth.models.faraday.new_gmm.new_gmm_model import GaussianMixtureModel


gmm_init_params = initialise_gmm_params(
    X=input_data,
    n_components = N_COMPONENTS,
    reg_covar=REG_COVAR,
)
torch_gmm = GaussianMixtureModel(
    num_components=N_COMPONENTS,
    num_features = input_data.shape[1],
    reg_covar=REG_COVAR,
    print_idx=IDX
)
torch_gmm.initialise(gmm_init_params)
trained_model = training_loop(model=torch_gmm, data=input_tensor, max_iter=EPOCHS)

 80%|████████  | 20/25 [00:05<00:01,  3.82it/s]

Converged: True. Number of iterations: 20


# SK Learn GMM Manual

In [12]:
import numpy as np
from scipy.special import logsumexp
from scipy import linalg

def _estimate_gaussian_parameters(X, resp, reg_covar=REG_COVAR):
    nk = resp.sum(axis=0) + 10 * np.finfo(resp.dtype).eps
    means = np.dot(resp.T, X) / nk[:, np.newaxis]
    n_components, n_features = means.shape
    covariances = np.empty((n_components, n_features, n_features))
    for k in range(n_components):
        diff = X - means[k]
        covariances[k] = np.dot(resp[:, k] * diff.T, diff) / nk[k]
        covariances[k].flat[:: n_features + 1] += reg_covar
    return nk, means, covariances

def _compute_precision_cholesky(covariances):
    estimate_precision_error_message = (
        "Fitting the mixture model failed because some components have "
        "ill-defined empirical covariance (for instance caused by singleton "
        "or collapsed samples). Try to decrease the number of components, "
        "or increase reg_covar."
    )

    n_components, n_features, _ = covariances.shape
    precisions_chol = np.empty((n_components, n_features, n_features))
    for k, covariance in enumerate(covariances):
        try:
            cov_chol = linalg.cholesky(covariance, lower=True)
        except linalg.LinAlgError:
            raise ValueError(estimate_precision_error_message)
        precisions_chol[k] = linalg.solve_triangular(
            cov_chol, np.eye(n_features), lower=True
        ).T
    return precisions_chol

def _compute_log_det_cholesky(matrix_chol, n_features):
    n_components, _, _ = matrix_chol.shape
    log_det_chol = np.sum(
        np.log(matrix_chol.reshape(n_components, -1)[:, :: n_features + 1]), 1
    )
    return log_det_chol

def _estimate_log_gaussian_prob(X, means, precisions_chol):
    n_samples, n_features = X.shape
    n_components, _ = means.shape

    log_det = _compute_log_det_cholesky(precisions_chol, n_features)

    log_prob = np.empty((n_samples, n_components))
    for k, (mu, prec_chol) in enumerate(zip(means, precisions_chol)):
        y = np.dot(X, prec_chol) - np.dot(mu, prec_chol)
        log_prob[:, k] = np.sum(np.square(y), axis=1)
    return -0.5 * (n_features * np.log(2 * np.pi) + log_prob) + log_det

def _estimate_log_weights(weights):
        return np.log(weights)

def _estimate_weighted_log_prob(X, means, precisions_chol, weights):
        return _estimate_log_gaussian_prob(X, means, precisions_chol) + _estimate_log_weights(weights)


def _estimate_log_prob_resp(X, means, precisions_chol, weights):
    weighted_log_prob = _estimate_weighted_log_prob(X, means, precisions_chol, weights)
    log_prob_norm = logsumexp(weighted_log_prob, axis=1)
    with np.errstate(under="ignore"):
        log_resp = weighted_log_prob - log_prob_norm[:, np.newaxis]
    return log_prob_norm, log_resp

def _e_step(X,means, precisions_chol, weights):
    log_prob_norm, log_resp = _estimate_log_prob_resp(X, means, precisions_chol, weights)
    return np.mean(log_prob_norm), log_resp

def _m_step(X, log_reponsibilities, reg_covar=REG_COVAR):

    weights_, means_, covariances_ = _estimate_gaussian_parameters(X,np.exp(log_reponsibilities),reg_covar=reg_covar)
    weights_ /= weights_.sum()

    precision_cholesky_ = _compute_precision_cholesky(covariances=covariances_)

    return precision_cholesky_, weights_, means_, covariances_

In [13]:
means = gmm_init_params["means"].detach().numpy()
weights = gmm_init_params["weights"].detach().numpy()
prec_chol = gmm_init_params["precision_cholesky"].detach().numpy()

print(f"Initial prec chol: {prec_chol[IDX][0][0]}. Initial mean: {means[IDX][0]}")

converged = False
lower_bound = -np.inf

for i in range(EPOCHS):
    prev_lower_bound = lower_bound

    print(f"Old Prec Chol: {prec_chol[IDX][0][0]}. Old means: {means[IDX][0]}")
    log_prob, log_resp = _e_step(input_data, means, prec_chol, weights)
    prec_chol, weights, means, covar = _m_step(input_data, log_resp)

    print(f"New prec chol: {prec_chol[IDX][0][0]}. New means: {means[IDX][0]}")

    # Converegence
    lower_bound = log_prob
    change = abs(lower_bound - prev_lower_bound)
    print(f"Change: {change}")
    if change < CONVERGENCE_TOL:
        converged = True
        break

print(f'Converged: {converged}. Number of iterations: {i}')

Initial prec chol: 4.167169570922852. Initial mean: 0.18039406836032867
Old Prec Chol: 4.167169570922852. Old means: 0.18039406836032867
New prec chol: 4.233167242590703. New means: 0.1601667862775156
Change: inf
Old Prec Chol: 4.233167242590703. Old means: 0.1601667862775156
New prec chol: 4.302608211404412. New means: 0.13654736888632887
Change: 0.5965802676941183
Old Prec Chol: 4.302608211404412. Old means: 0.13654736888632887
New prec chol: 4.170436562212954. New means: 0.11592101725647623
Change: 0.23705405366639276
Old Prec Chol: 4.170436562212954. Old means: 0.11592101725647623
New prec chol: 3.935564560399703. New means: 0.0963309089718066
Change: 0.15404550009157947
Old Prec Chol: 3.935564560399703. Old means: 0.0963309089718066
New prec chol: 3.804755944723549. New means: 0.08183459658313541
Change: 0.11730303257668773
Old Prec Chol: 3.804755944723549. Old means: 0.08183459658313541
New prec chol: 3.7495639777909733. New means: 0.06969489063703525
Change: 0.09750155304139752


# SK Learn GMM Epoch

In [14]:
from sklearn.mixture import GaussianMixture

init_weights = gmm_init_params["weights"]
init_means = gmm_init_params["means"]

skgmm = GaussianMixture(n_components=N_COMPONENTS, covariance_type='full', tol=CONVERGENCE_TOL, max_iter=EPOCHS, random_state=0, means_init = init_means, weights_init=init_weights, verbose=2, verbose_interval=1)
skgmm.fit(input_data)
skgmm_pred = skgmm.predict(input_data)

Initialization 0
  Iteration 1	 time lapse 4.62847s	 ll change inf
  Iteration 2	 time lapse 1.85284s	 ll change 0.55613
  Iteration 3	 time lapse 2.02959s	 ll change 0.24395
  Iteration 4	 time lapse 2.55099s	 ll change 0.16258
  Iteration 5	 time lapse 1.87178s	 ll change 0.12128
  Iteration 6	 time lapse 2.08313s	 ll change 0.10104
  Iteration 7	 time lapse 2.50615s	 ll change 0.08843
  Iteration 8	 time lapse 1.86207s	 ll change 0.07718
  Iteration 9	 time lapse 2.06804s	 ll change 0.05981
  Iteration 10	 time lapse 2.38199s	 ll change 0.04918
  Iteration 11	 time lapse 3.36751s	 ll change 0.04353
  Iteration 12	 time lapse 2.49519s	 ll change 0.03475
  Iteration 13	 time lapse 3.39737s	 ll change 0.02870
  Iteration 14	 time lapse 2.85775s	 ll change 0.02447
  Iteration 15	 time lapse 2.45130s	 ll change 0.02136
  Iteration 16	 time lapse 2.24533s	 ll change 0.01721
  Iteration 17	 time lapse 2.86297s	 ll change 0.01443
  Iteration 18	 time lapse 2.75476s	 ll change 0.01313
  Iter

In [15]:
skgmm.converged_, skgmm.n_iter_

(True, 21)

# Torch Lightning

In [16]:
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning import LightningDataModule
class CustomDataset(Dataset):
    def __init__(self, data_tensor: torch.Tensor):
        self.data = data_tensor
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]
    
class CustomDataModule(LightningDataModule):
    def __init__(self, data_tensor: torch.Tensor, batch_size: int):
        super().__init__()
        self.data_tensor = data_tensor
        self.batch_size = batch_size
    def setup(self, stage=""):
        self.custom_ds = CustomDataset(self.data_tensor)
    def train_dataloader(self):
        return DataLoader(self.custom_ds, batch_size=self.batch_size, shuffle=False, generator=g, worker_init_fn=seed_worker)
    
custom_dm = CustomDataModule(data_tensor=input_tensor, batch_size=25000)
custom_dm.setup(stage="")

In [17]:

for i in range(5):
    print(next(iter(custom_dm.train_dataloader()))[0][0])

tensor(0.0195, grad_fn=<SelectBackward0>)
tensor(0.0195, grad_fn=<SelectBackward0>)
tensor(0.0195, grad_fn=<SelectBackward0>)
tensor(0.0195, grad_fn=<SelectBackward0>)
tensor(0.0195, grad_fn=<SelectBackward0>)


In [18]:
from opensynth.models.faraday.new_gmm.new_gmm_model import GaussianMixtureLightningModule
gmm_module = GaussianMixtureModel(
    num_components=N_COMPONENTS,
    num_features = input_data.shape[1],
    reg_covar=REG_COVAR,
    print_idx=IDX
)
gmm_module.initialise(gmm_init_params)
print(f"Initial prec chol: {gmm_module.precision_cholesky[IDX][0][0]}. Initial mean: {gmm_module.means[IDX][0]}")

gmm_lightning_module = GaussianMixtureLightningModule(
    gmm_module = gmm_module,
    vae_module = vae_model,
    num_components = gmm_module.num_components,
    num_features = gmm_module.num_features,
    reg_covar = gmm_module.reg_covar,
    convergence_tolerance = CONVERGENCE_TOL
)
trainer = pl.Trainer(max_epochs=EPOCHS, accelerator="cpu", deterministic=True )
trainer.fit(gmm_lightning_module, custom_dm)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/Users/charlotte.avery/.virtualenvs/OpenSynth-BNsxhSIM/lib/python3.11/site-packages/pytorch_lightning/core/opt

Initial prec chol: 4.167169570922852. Initial mean: 0.18039406836032867
Epoch 20: 100%|██████████| 1/1 [00:00<00:00,  2.95it/s, v_num=20]


# Compare

In [19]:
IDX = 0

In [20]:
df_compare_means = pd.DataFrame()
df_compare_means["skgmm"] = skgmm.means_[IDX]
df_compare_means["numpy"] = means[IDX]
df_compare_means["torch"] = trained_model.means[IDX]
df_compare_means["lightning"] = gmm_lightning_module.gmm_module.means[IDX]
df_compare_means

,skgmm,numpy,torch,lightning
0,-0.040410,-0.015153,-0.003848,-0.003848
1,-1.593916,-1.652623,-1.620407,-1.620407
2,-0.846162,-0.902899,-0.916207,-0.916207
3,1.532826,1.634990,1.643758,1.643758
4,0.525577,0.576219,0.613053,0.613053
5,-0.086819,-0.078370,-0.074405,-0.074405
6,-0.543287,-0.597693,-0.581053,-0.581053
7,3.280627,3.354957,3.314699,3.314699
8,-0.262851,-0.284911,-0.289358,-0.289358
9,-1.726783,-1.837890,-1.762655,-1.762655


In [21]:
gmm_init_params["means"][IDX]

tensor([ 0.1804, -2.7538, -1.0911,  1.7346,  0.2788, -0.0517, -0.9778,  3.2770,
        -0.2799, -2.2817, -0.6563, -1.5545, -1.2570,  0.9383,  0.9554, -2.0788,
         5.9050,  3.9497])

In [22]:
df_compare_covar = pd.DataFrame()
df_compare_covar["skgmm"] = skgmm.covariances_[IDX][0]
df_compare_covar["numpy"] = covar[IDX][0]
df_compare_covar["torch"] = trained_model.covariances.detach().numpy()[IDX][0]
df_compare_covar["lightning"] = gmm_lightning_module.gmm_module.covariances.detach().numpy()[IDX][0]
df_compare_covar

,skgmm,numpy,torch,lightning
0,0.075478,0.067608,0.077627,0.077627
1,-0.194839,-0.185708,-0.222430,-0.222430
2,-0.110896,-0.103423,-0.099270,-0.099270
3,0.111483,0.109388,0.098755,0.098755
4,-0.019119,-0.022431,-0.008625,-0.008625
5,0.014319,0.008235,0.015343,0.015343
6,-0.137716,-0.139382,-0.117061,-0.117061
7,-0.080565,-0.067839,-0.083691,-0.083691
8,-0.017084,-0.017829,-0.011237,-0.011237
9,-0.040579,-0.060229,-0.044885,-0.044885


In [23]:
df_compare_pre_chol = pd.DataFrame()
df_compare_pre_chol["skgmm"] = skgmm.precisions_cholesky_[IDX][0]
df_compare_pre_chol["numpy"] = prec_chol[IDX][0]
df_compare_pre_chol["torch"] = trained_model.precision_cholesky.detach().numpy()[IDX][0]
df_compare_pre_chol["lightning"] = gmm_lightning_module.gmm_module.precision_cholesky.detach().numpy()[IDX][0]
df_compare_pre_chol

,skgmm,numpy,torch,lightning
0,3.639912,3.845917,3.589172,3.589172
1,2.479631,2.352261,2.467333,2.467333
2,3.041528,3.508878,2.368087,2.368087
3,-1.246541,-1.232897,-1.053467,-1.053467
4,-2.704551,-2.702441,-2.776443,-2.776443
5,-5.834039,-4.068906,-4.021445,-4.021445
6,3.075886,3.824865,3.385015,3.385015
7,12.186112,12.503750,12.140152,12.140152
8,0.430341,1.813504,0.944585,0.944585
9,-5.943625,-6.014369,-4.569670,-4.569670


In [24]:
df_compare_weights = pd.DataFrame()
df_compare_weights["skgmm"] = skgmm.weights_[:10]
df_compare_weights["numpy"] = weights[:10]
df_compare_weights["torch"] = trained_model.weights[:10]
df_compare_weights["lightning"] = gmm_lightning_module.gmm_module.weights.detach().numpy()[:10]
df_compare_weights

,skgmm,numpy,torch,lightning
0,0.004700,0.004545,0.004363,0.004363
1,0.000160,0.000160,0.000160,0.000160
2,0.029844,0.041581,0.032934,0.032934
3,0.021606,0.022298,0.019733,0.019733
4,0.002576,0.002501,0.002617,0.002617
5,0.000160,0.000160,0.000160,0.000160
6,0.006069,0.005619,0.006260,0.006260
7,0.004960,0.005710,0.005611,0.005611
8,0.000600,0.000600,0.000600,0.000600
9,0.000480,0.000480,0.000480,0.000480


# Sampling

In [25]:
def sample(means_, covariances_, weights_, n_samples):
    rng = np.random.RandomState(RANDOM_STATE)
    n_samples_comp = rng.multinomial(n_samples, weights_)

    X = np.vstack(
            [
                rng.multivariate_normal(mean, covariance, int(sample))
                for (mean, covariance, sample) in zip(
                    means_, covariances_, n_samples_comp
                )
            ]
        )
    
    y = np.concatenate(
        [np.full(sample, j, dtype=int) for j, sample in enumerate(n_samples_comp)]
    )
    return (X, y)

In [26]:
N_SAMPLES = 250

In [27]:
skgmm_samples = sample(skgmm.means_, skgmm.covariances_, skgmm.weights_, n_samples = N_SAMPLES)

skgmm_X, skgmm_y = skgmm_samples
skgmm_X[IDX], skgmm_y[IDX]

(array([ 0.06546331, -1.45815873, -0.33128334,  1.29380454,  1.67838391,
         0.08892928,  0.15957208,  1.80299064, -0.12120419, -2.49050242,
        -0.13582103, -1.17386479, -1.5188634 ,  0.41852592,  2.42205194,
        -1.77503955,  2.07498493,  2.61665642]),
 0)

In [28]:
samples = sample(means, covar, weights, n_samples = N_SAMPLES)

X, y = samples
X[IDX], y[IDX]

(array([ 0.01900764, -1.16245489, -0.77577511,  2.14979001,  2.07757197,
         0.07693889, -0.28857952,  2.92392678, -0.0494064 , -4.05625911,
        -0.67460345, -0.67508589, -2.63844236,  0.3934814 ,  3.08956995,
        -2.02227899,  6.05678877,  4.13936513]),
 0)

In [29]:
train_model_samples = sample(trained_model.means.detach().numpy(), trained_model.covariances.detach().numpy(), trained_model.weights.detach().numpy(), n_samples = N_SAMPLES)
train_model_X, train_model_y = train_model_samples
train_model_X[IDX], train_model_y[IDX]

/var/folders/w2/2x1gj41j1xb5z153q5m_4rn80000gn/T/ipykernel_99151/1110536824.py:7: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  rng.multivariate_normal(mean, covariance, int(sample))


(array([ 0.03574808, -2.79003573, -0.76025084,  0.82357755, -0.20516164,
        -0.03460098, -0.07782398,  3.9214106 , -0.22837557,  0.32822871,
         0.69329133, -2.19083986, -2.50094329,  2.28448515,  0.32796495,
        -1.06755265, 12.07750982,  3.10206355]),
 0)

In [30]:
gmm_lightning_samples = sample(gmm_lightning_module.gmm_module.means.detach().numpy(), gmm_lightning_module.gmm_module.covariances.detach().numpy(), gmm_lightning_module.gmm_module.weights.detach().numpy(), n_samples = N_SAMPLES)
gmm_lightning_X, gmm_lightning_y = train_model_samples
gmm_lightning_X[IDX], gmm_lightning_y[IDX]

/var/folders/w2/2x1gj41j1xb5z153q5m_4rn80000gn/T/ipykernel_99151/1110536824.py:7: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  rng.multivariate_normal(mean, covariance, int(sample))


(array([ 0.03574808, -2.79003573, -0.76025084,  0.82357755, -0.20516164,
        -0.03460098, -0.07782398,  3.9214106 , -0.22837557,  0.32822871,
         0.69329133, -2.19083986, -2.50094329,  2.28448515,  0.32796495,
        -1.06755265, 12.07750982,  3.10206355]),
 0)